# Lab 4 - Hybrid Search of title embeddings and full-text fields in Opensearch

In [1]:
require 'opensearch'
require 'transformers-rb'
require 'tqdm'
require 'date'
require 'json'
require 'polars-df'

true

In [2]:
# https://github.com/opensearch-project/opensearch-ruby
host = 'ruby-opensearch-node'
port = 9200

$client = OpenSearch::Client.new(hosts: [{ host: host, port: port }])
info = $client.info
puts "Welcome to #{info['version']['distribution']} #{info['version']['number']}!"

Welcome to opensearch 2.18.0!


## Use the same model and method to get the query embedding, with some defaults changed
Remember, the model is `intfloat/e5-small-v2` and we need to prefix any query with 'query:'

In [22]:
# The E5 models expect 'query:' and 'passage:' prefixes
$model = Transformers.pipeline("embedding", 'intfloat/e5-small-v2')

def get_embeddings(texts, prefix: "query: ", progress: false)
  # puts texts
  texts = [texts] unless texts.is_a?(Array)
  total = texts.length
  embeddings = []
  
  texts.each_with_index do |text, i|
    prefixed_text = "#{prefix}#{text}"
    embedding = $model.(prefixed_text)
    embeddings << embedding
    
    if progress
        percent = ((i + 1).to_f / total * 100).to_i
        print "\rProcessing embeddings: #{percent}% (#{i + 1}/#{total})"
    end
  end
  if progress
      print "\nDone!\n"
  end
  embeddings
end

:get_embeddings

## Define different query types
Here we define Opensearch query bodies for:
 - BM25
 - KNN
 - Hybrid

In [4]:
def get_bm25_body(querystring)
    {
      "query" => {
        "bool" => {
          "should" => [
            {
              "multi_match" => {
                "query" => querystring,
                "type" => "cross_fields",
                "fields" => ["description"],
                "boost" => 1.0
              }
            },
            {
              "multi_match" => {
                "query" => querystring,
                "type" => "cross_fields",
                "fields" => ["title"],
                "boost" => 1.1
              }
            },
            {
              "multi_match" => {
                "query" => querystring,
                "type" => "cross_fields",
                "fields" => ["title_exactish"],
                "boost" => 1.2
              }
            }
          ]
        }
      },
      "_source" => {"exclude" => ["title_embedding"]}
    }
end

:get_bm25_body

In [5]:
def get_knn_body(querystring)
    embeddings = get_embeddings(querystring)
    {
      "query" => {
        "bool" => {
          "should" => [
            {
              "knn" => {
                "title_embedding" => {
                  "vector" => embeddings[0],
                  "k" => 20
                }
              }
            }
          ]
        }
      },
      "_source" => {"exclude" => ["title_embedding"]}
    }
end

:get_knn_body

## Hybrid Query

In [6]:
def get_hybrid_body(querystring)
    embeddings = get_embeddings(querystring)
    {
      "query" => {
        "hybrid" => {
          "queries" => [
            {
              "bool" => {
                "should" => [
                  {
                    "multi_match" => {
                      "query" => querystring,
                      "type" => "cross_fields",
                      "fields" => ["description"],
                      "boost" => 1.0
                    }
                  },
                  {
                    "multi_match" => {
                      "query" => querystring,
                      "type" => "cross_fields",
                      "fields" => ["title"],
                      "boost" => 1.1
                    }
                  },
                  {
                    "multi_match" => {
                      "query" => querystring,
                      "type" => "cross_fields",
                      "fields" => ["title_exactish"],
                      "boost" => 1.2
                    }
                  }
                ]
              }        
            },
            {
              "knn" => {
                "title_embedding" => {
                  "vector" => embeddings[0],
                  "k" => 100
                }
              }
            }
          ]
        }
      },
      "_source" => {"exclude" => ["title_embedding"]}
    }
end

:get_hybrid_body

In [18]:
def serps(querystring, resp, k: 5, show: true)
    # this doesn't match the value in the other notebook.
    count = resp["hits"]["total"]["value"]
    if show
        puts resp
    end
    results = resp["hits"]["hits"]
    
    html_str = "<h4>Showing #{count} Results for <em>#{querystring}</em></h4><ol>"
    
    results[0...k].each do |result|
        score = result["_score"]
        title = result["_source"]["title"] || "No title"
        url = result["_source"]["url"] || "No URL"
        description = result["_source"]["description"]
        text = result["_source"]["text"] || ""
        snippet = description || "#{text[0...140]}..."
        
        html_str += "<li><b>#{title}</b>(#{score})<br>#{snippet}<br>"
        html_str += "<span style=\"font-size:0.8em\"><a href=\"#{url}\">#{url}</a></span></li>"
    end
    
    html_str += "</ol>"
    
    if show
        IRuby.display(IRuby.html(html_str))
    else
        html_str
    end
end

:serps

In [9]:
def make_normalization_pipeline(name, bm25_weight: 0.5, knn_weight: 0.5)
  body = {
    "description" => "Post processor for hybrid search with bm25=#{bm25_weight} and knn=#{knn_weight}",
    "phase_results_processors" => [
      {
        "normalization-processor" => {
          "normalization" => {
            "technique" => "min_max"
          },
          "combination" => {
            "technique" => "arithmetic_mean",
            "parameters" => {
              "weights" => [
                bm25_weight,
                knn_weight
              ]
            }
          }
        }
      }
    ]
  }
  
  # Changed to use positional parameters instead of named parameters
  resp = $client.transport.perform_request(
    "PUT",                              # method
    "/_search/pipeline/#{name}",        # path
    {},                                 # params
    body                                # body
  )
end

:make_normalization_pipeline

In [10]:
make_normalization_pipeline("nlp-search-pipeline-equal", bm25_weight: 0.5, knn_weight: 0.5)
make_normalization_pipeline("nlp-search-pipeline-bm25-heavy", bm25_weight: 0.6, knn_weight: 0.4)
make_normalization_pipeline("nlp-search-pipeline-knn-heavy", bm25_weight: 0.4, knn_weight: 0.6)

#<OpenSearch::Transport::Transport::Response:0x00002aaadff4b5a0 @status=200, @body={"acknowledged"=>true}, @headers={"content-type"=>"application/json; charset=UTF-8", "content-encoding"=>"gzip", "content-length"=>"47"}>

In [11]:
# def search(querystring, body, pipeline: "nlp-search-pipeline-equal", show: true)
#     resp = $client.search(body: body, index: "ai-search", params: {"search_pipeline" => pipeline})
#     serps(querystring, resp) if show
#     resp
# end

def search(querystring, body, pipeline: "nlp-search-pipeline-equal", show: true)
  resp = $client.search({
    index: "ai-search",
    body: body,
    search_pipeline: pipeline  # Moved from params into main options hash
  })
  serps(querystring, resp) if show
  resp
end

def search_bm25(querystring, show: true)
    body = get_bm25_body(querystring)
    search(querystring, body, show: show)
end

def search_knn(querystring, show: true)
    body = get_knn_body(querystring)
    search(querystring, body, show: show)
end

# def search_hybrid(querystring, pipeline: "nlp-search-pipeline-equal", show: true)
#     body = get_hybrid_body(querystring)
#     search(querystring, body, pipeline: pipeline, show: show)
# end

def search_hybrid(querystring, pipeline: "nlp-search-pipeline-equal", show: true)
  body = get_hybrid_body(querystring)
  search(querystring, body, pipeline: pipeline, show: show)
end

:search_hybrid

In [12]:
resp = search_hybrid("crypto scandal")

Processing embeddings: 100% (1/1)
Done!
{"took"=>50, "timed_out"=>false, "_shards"=>{"total"=>1, "successful"=>1, "skipped"=>0, "failed"=>0}, "hits"=>{"total"=>{"value"=>935, "relation"=>"eq"}, "max_score"=>0.5, "hits"=>[{"_index"=>"ai-search", "_id"=>"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881", "_score"=>0.5, "_source"=>{"date"=>nil, "image_url"=>"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881", "domain"=>"www.sparkspread.com", "description"=>"Real time energy financing and trading news", "text"=>"June 15, 2017 00:21 EST\nWelcome to SparkSpread.com\nThis page is only available to subscribers.\nIf you are already a subscriber, please log in below...\nLog in\nIf you would like to register for a free trial to SparkSpread.com, click here", "title"=>"Real time energy financing and trading news", "url"=>"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fs

"<h4>Showing 935 Results for <em>crypto scandal</em></h4><ol><li><b>Real time energy financing and trading news</b>(0.5)<br>Real time energy financing and trading news<br><span style=\"font-size:0.8em\"><a href=\"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881\">http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881</a></span></li><li><b>Real time energy financing and trading news</b>(0.5)<br>Real time energy financing and trading news<br><span style=\"font-size:0.8em\"><a href=\"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20254\">http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20254</a></span></li><li><b>Real time energy financing and trading news</b>(0.5)<br>Real time energy financing and trading news<br><span style=\"font-size:0.8em\"><a href=\"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878\">http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878</a></span></li><li><b>Real time energy financing and trading news</b>(0.5)<br>Real time energy financing and trading news<br><span style=\"font-size:0.8em\"><a href=\"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20879\">http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20879</a></span></li><li><b>Putin to athletes: Ignore doping scandals at Olympics</b>(0.5)<br>Putin to athletes: Ignore doping scandals at Olympics<br><span style=\"font-size:0.8em\"><a href=\"https://www.taiwannews.com.tw/en/news/3354788\">https://www.taiwannews.com.tw/en/news/3354788</a></span></li></ol>"

{"took"=>50, "timed_out"=>false, "_shards"=>{"total"=>1, "successful"=>1, "skipped"=>0, "failed"=>0}, "hits"=>{"total"=>{"value"=>935, "relation"=>"eq"}, "max_score"=>0.5, "hits"=>[{"_index"=>"ai-search", "_id"=>"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881", "_score"=>0.5, "_source"=>{"date"=>nil, "image_url"=>"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881", "domain"=>"www.sparkspread.com", "description"=>"Real time energy financing and trading news", "text"=>"June 15, 2017 00:21 EST\nWelcome to SparkSpread.com\nThis page is only available to subscribers.\nIf you are already a subscriber, please log in below...\nLog in\nIf you would like to register for a free trial to SparkSpread.com, click here", "title"=>"Real time energy financing and trading news", "url"=>"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881"

In [13]:
resp = search_bm25("crypto scandal")

{"took"=>10, "timed_out"=>false, "_shards"=>{"total"=>1, "successful"=>1, "skipped"=>0, "failed"=>0}, "hits"=>{"total"=>{"value"=>2, "relation"=>"eq"}, "max_score"=>23.372225, "hits"=>[{"_index"=>"ai-search", "_id"=>"https://www.taiwannews.com.tw/en/news/3354788", "_score"=>23.372225, "_source"=>{"date"=>"2018-01-31T21:08:00", "image_url"=>"https://tnimage.taiwannews.com.tw/photos/shares/AP/20180203/95534fa56fb24671a669c62c3da71149.jpg", "domain"=>"www.taiwannews.com.tw", "description"=>"Putin to athletes: Ignore doping scandals at Olympics", "text"=>"MOSCOW (AP) — Russian President Vladimir Putin has told the country's athletes to forget about doping scandals when they compete at the Pyeongchang Olympics.\nAs punishment for what it deemed a doping scheme during the 2014 Sochi Olympics, the International Olympic Committee has invited 169 Russians to compete under a neutral flag using the name \"Olympic Athletes from Russia.\"\nAt a meeting with athletes at his presidential residence ou

"<h4>Showing 2 Results for <em>crypto scandal</em></h4><ol><li><b>Putin to athletes: Ignore doping scandals at Olympics</b>(23.372225)<br>Putin to athletes: Ignore doping scandals at Olympics<br><span style=\"font-size:0.8em\"><a href=\"https://www.taiwannews.com.tw/en/news/3354788\">https://www.taiwannews.com.tw/en/news/3354788</a></span></li><li><b>More than 4,000 students and teachers to be compensated for Taiwan food safety scandal</b>(12.074548)<br>Tainted oil from Chang Guann turned up in food at 22 schools in 2014.A total of 4,048 students and teachers can soon expect to receive between NT$3,000 (US$98) and NT$9,000 each as compensation for the consumption of tainted cooking oil from Chang Guann.<br><span style=\"font-size:0.8em\"><a href=\"https://www.taiwannews.com.tw/en/news/3473912\">https://www.taiwannews.com.tw/en/news/3473912</a></span></li></ol>"

{"took"=>10, "timed_out"=>false, "_shards"=>{"total"=>1, "successful"=>1, "skipped"=>0, "failed"=>0}, "hits"=>{"total"=>{"value"=>2, "relation"=>"eq"}, "max_score"=>23.372225, "hits"=>[{"_index"=>"ai-search", "_id"=>"https://www.taiwannews.com.tw/en/news/3354788", "_score"=>23.372225, "_source"=>{"date"=>"2018-01-31T21:08:00", "image_url"=>"https://tnimage.taiwannews.com.tw/photos/shares/AP/20180203/95534fa56fb24671a669c62c3da71149.jpg", "domain"=>"www.taiwannews.com.tw", "description"=>"Putin to athletes: Ignore doping scandals at Olympics", "text"=>"MOSCOW (AP) — Russian President Vladimir Putin has told the country's athletes to forget about doping scandals when they compete at the Pyeongchang Olympics.\nAs punishment for what it deemed a doping scheme during the 2014 Sochi Olympics, the International Olympic Committee has invited 169 Russians to compete under a neutral flag using the name \"Olympic Athletes from Russia.\"\nAt a meeting with athletes at his presidential residence ou

In [14]:
resp = search_knn("crypto scandal")

Processing embeddings: 100% (1/1)
Done!
{"took"=>4, "timed_out"=>false, "_shards"=>{"total"=>1, "successful"=>1, "skipped"=>0, "failed"=>0}, "hits"=>{"total"=>{"value"=>189, "relation"=>"eq"}, "max_score"=>0.90931046, "hits"=>[{"_index"=>"ai-search", "_id"=>"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878", "_score"=>0.90931046, "_source"=>{"date"=>nil, "image_url"=>"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878", "domain"=>"www.sparkspread.com", "description"=>"Real time energy financing and trading news", "text"=>"June 14, 2017 16:48 EST\nWelcome to SparkSpread.com\nThis page is only available to subscribers.\nIf you are already a subscriber, please log in below...\nLog in\nIf you would like to register for a free trial to SparkSpread.com, click here", "title"=>"Real time energy financing and trading news", "url"=>"http://www.sparkspread.com/login.php?return=%2Fsparks

"<h4>Showing 189 Results for <em>crypto scandal</em></h4><ol><li><b>Real time energy financing and trading news</b>(0.90931046)<br>Real time energy financing and trading news<br><span style=\"font-size:0.8em\"><a href=\"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878\">http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878</a></span></li><li><b>Real time energy financing and trading news</b>(0.90931046)<br>Real time energy financing and trading news<br><span style=\"font-size:0.8em\"><a href=\"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20879\">http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20879</a></span></li><li><b>Real time energy financing and trading news</b>(0.90931046)<br>Real time energy financing and trading news<br><span style=\"font-size:0.8em\"><a href=\"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881\">http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881</a></span></li><li><b>Real time energy financing and trading news</b>(0.90931046)<br>Real time energy financing and trading news<br><span style=\"font-size:0.8em\"><a href=\"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20254\">http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20254</a></span></li><li><b>সোশ্যাল মিডিয়ায় ছবি দিয়ে কি বয়ফ্রেন্ডের কথাই সবাইকে জানাতে চাইছেন সোনম?</b>(0.9087945)<br>There have been rumours that they might get hitched in 2017, but the Neerja actor has always kept mum on this topic<br><span style=\"font-size:0.8em\"><a href=\"http://banglalive.com/is-sonam-kapoor-finally-making-her-relationship-with-anand-ahuja-public-with-this-pic/\">http://banglalive.com/is-sonam-kapoor-finally-making-her-relationship-with-anand-ahuja-public-with-this-pic/</a></span></li></ol>"

{"took"=>4, "timed_out"=>false, "_shards"=>{"total"=>1, "successful"=>1, "skipped"=>0, "failed"=>0}, "hits"=>{"total"=>{"value"=>189, "relation"=>"eq"}, "max_score"=>0.90931046, "hits"=>[{"_index"=>"ai-search", "_id"=>"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878", "_score"=>0.90931046, "_source"=>{"date"=>nil, "image_url"=>"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878", "domain"=>"www.sparkspread.com", "description"=>"Real time energy financing and trading news", "text"=>"June 14, 2017 16:48 EST\nWelcome to SparkSpread.com\nThis page is only available to subscribers.\nIf you are already a subscriber, please log in below...\nLog in\nIf you would like to register for a free trial to SparkSpread.com, click here", "title"=>"Real time energy financing and trading news", "url"=>"http://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%

In [15]:
def search_compare(querystring, pipeline: "nlp-search-pipeline-equal")
    bm25 = serps(querystring, search_bm25(querystring, show: false), k: 5, show: false)
    knn = serps(querystring, search_knn(querystring, show: false), k: 5, show: false)
    hybrid = serps(querystring, search_hybrid(querystring, pipeline: pipeline, show: false), k: 5, show: false)
    
    html_all = <<-HTML
        <style>
            .compare li {overflow-x:hidden;width:320px!important;text-align:left;height:200px;border-bottom:1px solid #333;}
        </style>
        <table class="compare">
            <tr><td>BM25</td><td>KNN</td><td>Hybrid (#{pipeline})</td></tr>
            <tr><td>#{bm25}</td><td>#{knn}</td><td>#{hybrid}</td></tr>
        </table>
    HTML
    
    IRuby.display(IRuby.html(html_all))
end

:search_compare

In [23]:
search_compare("property market", pipeline: "nlp-search-pipeline-equal")

BM25,KNN,Hybrid (nlp-search-pipeline-equal)
"Showing 21 Results for property marketHow to use the Windows Tags property to manage Office files(19.945515)Searching, grouping, and even filtering files is easier if you know how to use the Windows Tags property.https://www.techrepublic.com/blog/microsoft-office/use-windows-tags-property-to-manage-office-files/Global GIS Market in Telecom Sector - Use of GIS and Big Data is an Emerging Trend in the Market(16.333023)Global GIS Market in Telecom Sector - Use of GIS and Big Data is an Emerging Trend in the Market | Technaviohttps://www.taiwannews.com.tw/en/news/3413697Top Factors Driving the Global Potassium Sulfate Market(14.88765)Top Factors Driving the Global Potassium Sulfate Market | Technaviohttps://www.taiwannews.com.tw/en/news/3413580Global Allergy Immunotherapies Market - Industry Analysis and Forecast(14.88765)Global Allergy Immunotherapies Market - Industry Analysis and Forecast | Technaviohttps://www.taiwannews.com.tw/en/news/3444398Technological Advances to Drive the Global Dental Handpieces Market(14.643564)Technological Advances to Drive the Global Dental Handpieces Market| Technaviohttps://www.taiwannews.com.tw/en/news/3444389","Showing 20 Results for property marketজেনের সঙ্গে আলাপ‚প্রেম থেকে বিয়ে – অকপট প্রীতি জিন্টা(0.90857416)It was a mystery and shocking to see Preity tying the knot suddenly as few people knew that she is dating Gene.http://banglalive.com/preity-zinta-reveals-where-she-first-met-hubby-gene-goodenough/ভ্যালেন্টাইনের হাত ধরে গভীর রাতে জুহু বিচে শাহরুখ(0.9081973)The King of romance, Shah Rukh Khan spent an amazing time when the Valentine’s day begun.http://banglalive.com/srk-goes-late-night-walk-valentine/প্রাক্তন প্রেমিকা প্রিয়াঙ্কাকে মেরে ফেলতে চান শাহিদ!!(0.9066849)The Kill Marry Hookup sequence on Koffee With Karan rapid fire round always puts the celebs in a tongue-tied situation.http://banglalive.com/shahid-wants-to-kill-priyanka-marry-sonakshi-and-hook-up-with-alia/Global GIS Market in Telecom Sector - Use of GIS and Big Data is an Emerging Trend in the Market(0.90544224)Global GIS Market in Telecom Sector - Use of GIS and Big Data is an Emerging Trend in the Market | Technaviohttps://www.taiwannews.com.tw/en/news/3413697Business Highlights(0.90523434)Business Highlightshttps://www.taiwannews.com.tw/en/news/3385834","Showing 109 Results for property marketHow to use the Windows Tags property to manage Office files(0.5)Searching, grouping, and even filtering files is easier if you know how to use the Windows Tags property.https://www.techrepublic.com/blog/microsoft-office/use-windows-tags-property-to-manage-office-files/জেনের সঙ্গে আলাপ‚প্রেম থেকে বিয়ে – অকপট প্রীতি জিন্টা(0.5)It was a mystery and shocking to see Preity tying the knot suddenly as few people knew that she is dating Gene.http://banglalive.com/preity-zinta-reveals-where-she-first-met-hubby-gene-goodenough/ভ্যালেন্টাইনের হাত ধরে গভীর রাতে জুহু বিচে শাহরুখ(0.4624092)The King of romance, Shah Rukh Khan spent an amazing time when the Valentine’s day begun.http://banglalive.com/srk-goes-late-night-walk-valentine/Global GIS Market in Telecom Sector - Use of GIS and Big Data is an Emerging Trend in the Market(0.4278494)Global GIS Market in Telecom Sector - Use of GIS and Big Data is an Emerging Trend in the Market | Technaviohttps://www.taiwannews.com.tw/en/news/3413697প্রাক্তন প্রেমিকা প্রিয়াঙ্কাকে মেরে ফেলতে চান শাহিদ!!(0.31155846)The Kill Marry Hookup sequence on Koffee With Karan rapid fire round always puts the celebs in a tongue-tied situation.http://banglalive.com/shahid-wants-to-kill-priyanka-marry-sonakshi-and-hook-up-with-alia/"


In [39]:
search_compare("property market", pipeline: "nlp-search-pipeline-bm25-heavy")

Processing embeddings: 100% (1/1)
Done!
Processing embeddings: 100% (1/1)
Done!


BM25,KNN,Hybrid (nlp-search-pipeline-bm25-heavy)
"Showing 21 Results for property marketHow to use the Windows Tags property to manage Office files(19.945515)Searching, grouping, and even filtering files is easier if you know how to use the Windows Tags property.https://www.techrepublic.com/blog/microsoft-office/use-windows-tags-property-to-manage-office-files/Global GIS Market in Telecom Sector - Use of GIS and Big Data is an Emerging Trend in the Market(16.333023)Global GIS Market in Telecom Sector - Use of GIS and Big Data is an Emerging Trend in the Market | Technaviohttps://www.taiwannews.com.tw/en/news/3413697Global Allergy Immunotherapies Market - Industry Analysis and Forecast(14.88765)Global Allergy Immunotherapies Market - Industry Analysis and Forecast | Technaviohttps://www.taiwannews.com.tw/en/news/3444398Top Factors Driving the Global Potassium Sulfate Market(14.88765)Top Factors Driving the Global Potassium Sulfate Market | Technaviohttps://www.taiwannews.com.tw/en/news/3413580Technological Advances to Drive the Global Dental Handpieces Market(14.643564)Technological Advances to Drive the Global Dental Handpieces Market| Technaviohttps://www.taiwannews.com.tw/en/news/3444389",Showing 20 Results for property marketBusiness Highlights(0.87082547)Business Highlightshttps://www.taiwannews.com.tw/en/news/3385834Business Highlights(0.87082547)Business Highlightshttp://www.taiwannews.com.tw/en/news/3095159Real time energy financing and trading news(0.8686811)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878Real time energy financing and trading news(0.8686811)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20879Real time energy financing and trading news(0.8686811)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881,"Showing 10 Results for property marketHow to use the Windows Tags property to manage Office files(0.6)Searching, grouping, and even filtering files is easier if you know how to use the Windows Tags property.https://www.techrepublic.com/blog/microsoft-office/use-windows-tags-property-to-manage-office-files/Business Highlights(0.4)Business Highlightshttp://www.taiwannews.com.tw/en/news/3095159Business Highlights(0.4)Business Highlightshttps://www.taiwannews.com.tw/en/news/3385834Real time energy financing and trading news(0.3799276)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878Real time energy financing and trading news(0.3799276)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20879"


In [40]:
search_compare("property market", pipeline: "nlp-search-pipeline-knn-heavy")

Processing embeddings: 100% (1/1)
Done!
Processing embeddings: 100% (1/1)
Done!


BM25,KNN,Hybrid (nlp-search-pipeline-knn-heavy)
"Showing 21 Results for property marketHow to use the Windows Tags property to manage Office files(19.945515)Searching, grouping, and even filtering files is easier if you know how to use the Windows Tags property.https://www.techrepublic.com/blog/microsoft-office/use-windows-tags-property-to-manage-office-files/Global GIS Market in Telecom Sector - Use of GIS and Big Data is an Emerging Trend in the Market(16.333023)Global GIS Market in Telecom Sector - Use of GIS and Big Data is an Emerging Trend in the Market | Technaviohttps://www.taiwannews.com.tw/en/news/3413697Global Allergy Immunotherapies Market - Industry Analysis and Forecast(14.88765)Global Allergy Immunotherapies Market - Industry Analysis and Forecast | Technaviohttps://www.taiwannews.com.tw/en/news/3444398Top Factors Driving the Global Potassium Sulfate Market(14.88765)Top Factors Driving the Global Potassium Sulfate Market | Technaviohttps://www.taiwannews.com.tw/en/news/3413580Technological Advances to Drive the Global Dental Handpieces Market(14.643564)Technological Advances to Drive the Global Dental Handpieces Market| Technaviohttps://www.taiwannews.com.tw/en/news/3444389",Showing 20 Results for property marketBusiness Highlights(0.87082547)Business Highlightshttps://www.taiwannews.com.tw/en/news/3385834Business Highlights(0.87082547)Business Highlightshttp://www.taiwannews.com.tw/en/news/3095159Real time energy financing and trading news(0.8686811)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878Real time energy financing and trading news(0.8686811)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20879Real time energy financing and trading news(0.8686811)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881,Showing 10 Results for property marketBusiness Highlights(0.6)Business Highlightshttp://www.taiwannews.com.tw/en/news/3095159Business Highlights(0.6)Business Highlightshttps://www.taiwannews.com.tw/en/news/3385834Real time energy financing and trading news(0.5698914)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878Real time energy financing and trading news(0.5698914)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20879Real time energy financing and trading news(0.5698914)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881


In [41]:
search_compare("crypto scandal")

Processing embeddings: 100% (1/1)
Done!
Processing embeddings: 100% (1/1)
Done!


BM25,KNN,Hybrid (nlp-search-pipeline-equal)
"Showing 2 Results for crypto scandalPutin to athletes: Ignore doping scandals at Olympics(22.885584)Putin to athletes: Ignore doping scandals at Olympicshttps://www.taiwannews.com.tw/en/news/3354788More than 4,000 students and teachers to be compensated for Taiwan food safety scandal(11.866891)Tainted oil from Chang Guann turned up in food at 22 schools in 2014.A total of 4,048 students and teachers can soon expect to receive between NT$3,000 (US$98) and NT$9,000 each as compensation for the consumption of tainted cooking oil from Chang Guann.https://www.taiwannews.com.tw/en/news/3473912","Showing 20 Results for crypto scandalReal time energy financing and trading news(0.88546175)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878Real time energy financing and trading news(0.88546175)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20879Real time energy financing and trading news(0.88546175)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881Real time energy financing and trading news(0.88546175)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20254আরিব্বাস, জাল ডাক্তার!(0.88258606)In recent past there has been many reported incidents of fake doctors being arrested from every nuke and corner of Bengal - a funny take on thathttp://banglalive.com/the-increasing-number-of-fake-doctors-and-their-treatment/",Showing 10 Results for crypto scandalReal time energy financing and trading news(0.5)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20878Real time energy financing and trading news(0.5)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20879Putin to athletes: Ignore doping scandals at Olympics(0.5)Putin to athletes: Ignore doping scandals at Olympicshttps://www.taiwannews.com.tw/en/news/3354788Real time energy financing and trading news(0.5)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20881Real time energy financing and trading news(0.5)Real time energy financing and trading newshttp://www.sparkspread.com/login.php?return=%2Fsparkspread.php%3Fsparkspread_concept%3Dstory%26id%3D20254


In [42]:
search_compare("US economic recovery")

Processing embeddings: 100% (1/1)
Done!
Processing embeddings: 100% (1/1)
Done!


BM25,KNN,Hybrid (nlp-search-pipeline-equal)
"Showing 82 Results for US economic recoveryTrump to herald economic progress in State of the Union(23.450245)Trump to herald economic progress in State of the Unionhttps://www.taiwannews.com.tw/en/news/3354102BC-US--Sugar, US(14.562808)BC-US--Sugar, UShttps://www.taiwannews.com.tw/en/news/3385764BC-US--Gold, US(14.562808)BC-US--Gold, UShttps://www.taiwannews.com.tw/en/news/3354953BC-US--Copper, US(14.562808)BC-US--Copper, UShttps://www.taiwannews.com.tw/en/news/3354438BC-US--Cocoa, US(14.562808)BC-US--Cocoa, UShttps://www.taiwannews.com.tw/en/news/3385769","Showing 20 Results for US economic recoveryTrump to herald economic progress in State of the Union(0.8791541)Trump to herald economic progress in State of the Unionhttps://www.taiwannews.com.tw/en/news/3354102How Fed hike will affect US consumers and overseas economies(0.8613023)How Fed hike will affect US consumers and overseas economieshttp://www.taiwannews.com.tw/en/news/3188359Text of President Donald Trump's State of the Union address(0.8606802)Text of President Donald Trump's State of the Union addresshttps://www.taiwannews.com.tw/en/news/3354631BC-US--Index, US(0.8571551)BC-US--Index, UShttps://www.taiwannews.com.tw/en/news/3318227AP NewsAlert(0.85561657)AP NewsAlerthttp://www.taiwannews.com.tw/en/news/3188148","Showing 10 Results for US economic recoveryTrump to herald economic progress in State of the Union(1.0)Trump to herald economic progress in State of the Unionhttps://www.taiwannews.com.tw/en/news/3354102How Fed hike will affect US consumers and overseas economies(0.3170116)How Fed hike will affect US consumers and overseas economieshttp://www.taiwannews.com.tw/en/news/3188359Text of President Donald Trump's State of the Union address(0.31063485)Text of President Donald Trump's State of the Union addresshttps://www.taiwannews.com.tw/en/news/3354631BC-US--Index, US(0.27450064)BC-US--Index, UShttps://www.taiwannews.com.tw/en/news/3318227AP NewsAlert(0.2587302)AP NewsAlerthttp://www.taiwannews.com.tw/en/news/3188148"
